<a href="https://colab.research.google.com/github/annab0503/DS2002/blob/main/Assignments/Data%20Project%201/DS2002_Data_Project_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Fetch/Download/Retrieve
Fetch / download / retrieve a remote data file by URL, or ingest a local
file mounted.

In [10]:
# Import Dependencies
import requests
import json
import pandas as pd
import sqlite3 as sql

In [11]:
# List of data sources
data_sources = [
    "https://raw.githubusercontent.com/annab0503/DS2002/refs/heads/main/Assignments/Data%20Project%201/Data%20Files/2023_Border_Crossing_Entry_Data.json",  # JSON File
    "https://raw.githubusercontent.com/annab0503/DS2002/refs/heads/main/Assignments/Data%20Project%201/Data%20Files/Crime_Incidents_in_2024.csv"  # CSV File
]

# Loop through the list of URLs
for url in data_sources:
    try:
        if url.endswith('.json'):
            json_df = pd.read_json(url)
            print("JSON DataFrame imported:")
            print(json_df.head())
            # Summary of ingestion
            print(f"Ingested JSON Data - Records: {len(json_df)}, Columns: {len(json_df.columns)}")

        elif url.endswith('.csv'):
            csv_df = pd.read_csv(url)
            print("CSV DataFrame imported:")
            print(csv_df.head())
            # Summary of ingestion
            print(f"Ingested CSV Data - Records: {len(csv_df)}, Columns: {len(csv_df.columns)}")

        else:
            print(f"File format not supported for URL: {url}")
    except Exception as e:
        print(f"Error importing data from {url}: {e}")

JSON DataFrame imported:
       Port Name    State  Port Code            Border       Date Measure  \
0           Roma    Texas       2310  US-Mexico Border 2023-12-01   Buses   
1        Del Rio    Texas       2302  US-Mexico Border 2023-12-01  Trucks   
2           Roma    Texas       2310  US-Mexico Border 2023-11-01  Trucks   
3        Douglas  Arizona       2601  US-Mexico Border 2023-10-01   Buses   
4  Beecher Falls  Vermont        206  US-Canada Border 2023-08-01  Trucks   

   Value  Latitude  Longitude                          Point  
0     46    26.404    -99.019   POINT (-99.018981 26.403928)  
1   6552    29.327   -100.928  POINT (-100.927612 29.326784)  
2   3753    26.404    -99.019   POINT (-99.018981 26.403928)  
3     13    31.334   -109.560  POINT (-109.560344 31.334043)  
4    422    45.013    -71.505   POINT (-71.505309 45.013411)  
Ingested JSON Data - Records: 9087, Columns: 10
CSV DataFrame imported:
             X            Y       CCN              REPORT_DAT 

## Modify
Modify the number of columns from the source to the destination,
reducing or adding columns. If you add data cols you can put any other
useful information in that column you wish.

### JSON File

In [4]:
# Split 'Date' column into 'Month', 'Day', and 'Year' columns
json_df['Month'] = json_df['Date'].dt.month
json_df['Day'] = json_df['Date'].dt.day
json_df['Year'] = json_df['Date'].dt.year

# Drop the original 'Date' column
json_df = json_df.drop(columns=['Date'])

# Drop the 'Year' column (This dataframe contains 2023 data only)
json_df = json_df.drop(columns=['Year'])

# Drop the Point column
json_df = json_df.drop(columns=['Point'])

# Rename 'Measure' column to 'Mode of Transportation' (more descriptive)
json_df = json_df.rename(columns={'Measure': 'Mode of Transportation'})

# Rename 'Value' column to 'Total Count' (more descriptive)
json_df = json_df.rename(columns={'Value': 'Total Count'})

# Remove the word "Border" from each value in the 'Border' column
json_df['Border'] = json_df['Border'].str.replace(' Border', '', regex=False)

#Re-order columns
json_df = json_df[['Month', 'Day', 'Port Code', 'Port Name', 'Border', 'Mode of Transportation', 'Total Count']]

In [5]:
# View modified dataframe
json_df

,Month,Day,Port Code,Port Name,Border,Mode of Transportation,Total Count
0,12,1,2310,Roma,US-Mexico,Buses,46
1,12,1,2302,Del Rio,US-Mexico,Trucks,6552
2,11,1,2310,Roma,US-Mexico,Trucks,3753
3,10,1,2601,Douglas,US-Mexico,Buses,13
4,8,1,206,Beecher Falls,US-Canada,Trucks,422
...,...,...,...,...,...,...,...
9082,11,1,3803,Sault Sainte Marie,US-Canada,Buses,25
9083,3,1,3401,Pembina,US-Canada,Truck Containers Loaded,17600
9084,3,1,3604,International Falls,US-Canada,Bus Passengers,310
9085,9,1,3416,Maida,US-Canada,Personal Vehicles,412


In [6]:
# Summary of post-processing for JSON DataFrame
print(f"Post-Processed JSON Data - Records: {len(json_df)}, Columns: {len(json_df.columns)}")

Post-Processed JSON Data - Records: 9087, Columns: 7


### CSV File

In [7]:
# Drop some columns
csv_df = csv_df.drop(columns=['X', 'Y', 'ANC', 'BLOCK', 'BLOCK_GROUP', 'XBLOCK', 'YBLOCK', 'WARD', 'DISTRICT', 'NEIGHBORHOOD_CLUSTER', 'CENSUS_TRACT', 'VOTING_PRECINCT', 'BID', 'OBJECTID', 'OCTO_RECORD_ID'])

# Convert columns to datetime format
csv_df['START_DATE'] = pd.to_datetime(csv_df['START_DATE'])
csv_df['END_DATE'] = pd.to_datetime(csv_df['END_DATE'])
csv_df['REPORT_DAT'] = pd.to_datetime(csv_df['REPORT_DAT'])

# Create new Date and Time columns
csv_df['Start Date'] = csv_df['START_DATE'].dt.date
csv_df['Start Time'] = csv_df['START_DATE'].dt.time

csv_df['End Date'] = csv_df['END_DATE'].dt.date
csv_df['End Time'] = csv_df['END_DATE'].dt.time

csv_df['Report Date'] = csv_df['REPORT_DAT'].dt.date
csv_df['Report Time'] = csv_df['REPORT_DAT'].dt.time

# Drop original columns
csv_df = csv_df.drop(['START_DATE', 'END_DATE', 'REPORT_DAT'], axis=1)

# Rename columns
csv_df = csv_df.rename(columns={'CCN': 'Criminal Complaint Number (CCN)'})
csv_df = csv_df.rename(columns={'PSA': 'Police Service Area (PSA)'})
csv_df = csv_df.rename(columns={'LATITUDE': 'Latitude (X)'})
csv_df = csv_df.rename(columns={'LONGITUDE': 'Longitude (Y)'})
csv_df = csv_df.rename(columns={'SHIFT': 'Shift'})
csv_df = csv_df.rename(columns={'OFFENSE': 'Offense'})
csv_df = csv_df.rename(columns={'METHOD': 'Method'})

# Re-order columns
csv_df = csv_df[['Start Date', 'Start Time', 'End Date', 'End Time', 'Report Date', 'Report Time', 'Criminal Complaint Number (CCN)', 'Police Service Area (PSA)', 'Shift', 'Latitude (X)', 'Longitude (Y)', 'Offense', 'Method']]

In [8]:
csv_df

,Start Date,Start Time,End Date,End Time,Report Date,Report Time,Criminal Complaint Number (CCN),Police Service Area (PSA),Shift,Latitude (X),Longitude (Y),Offense,Method
0,2024-03-21,11:55:00,2024-03-21,12:57:00,2024-03-21,14:26:00,24042662,607.0,DAY,38.867749,-76.979006,BURGLARY,OTHERS
1,2024-08-11,03:30:00,2024-08-14,04:43:00,2024-08-14,04:48:00,24124362,303.0,MIDNIGHT,38.921623,-77.042365,MOTOR VEHICLE THEFT,OTHERS
2,2024-06-14,02:00:00,2024-06-14,06:00:00,2024-06-14,07:58:03,24090303,506.0,MIDNIGHT,38.914214,-76.986030,THEFT F/AUTO,OTHERS
3,2024-06-19,16:43:00,2024-06-19,17:10:00,2024-06-19,17:29:11,24093316,505.0,DAY,38.921585,-76.993756,THEFT/OTHER,OTHERS
4,2024-03-07,10:55:00,2024-03-07,13:00:00,2024-03-07,13:11:19,24034913,103.0,DAY,38.880281,-77.015187,MOTOR VEHICLE THEFT,OTHERS
...,...,...,...,...,...,...,...,...,...,...,...,...,...
21456,2024-08-06,16:53:00,2024-08-06,16:53:00,2024-08-06,20:20:10,24120519,204.0,EVENING,38.924200,-77.051903,THEFT/OTHER,OTHERS
21457,2024-02-01,23:01:00,2024-02-01,23:52:00,2024-02-02,03:09:55,24016391,303.0,EVENING,38.923870,-77.040876,THEFT/OTHER,OTHERS
21458,2024-02-03,07:37:00,2024-02-03,07:39:00,2024-02-03,08:47:02,24017154,303.0,MIDNIGHT,38.917403,-77.041650,ASSAULT W/DANGEROUS WEAPON,KNIFE
21459,2024-02-06,22:00:00,2024-02-06,22:45:00,2024-02-17,00:05:34,24019047,107.0,EVENING,38.885134,-76.997315,THEFT/OTHER,OTHERS


In [9]:
# Summary of post-processing for CSV DataFrame
print(f"Post-Processed CSV Data - Records: {len(csv_df)}, Columns: {len(csv_df.columns)}")

Post-Processed CSV Data - Records: 21461, Columns: 13


## Convert/Store
Convert the general format and data structure of the data source to any target, and write the converted (new) file to disk (local file) or a SQL database.

In [17]:
# Choose your desired data source
df_choice = input("Which data source to export? (csv_df, json_df): ").strip().lower()

Which data source to export? (csv_df, json_df): json_df


In [18]:
# Select the appropriate data source
if df_choice == 'csv_df':
    df_to_export = csv_df
elif df_choice == 'json_df':
    df_to_export = json_df
else:
    print("Invalid choice. Please select 'csv_df' or 'json_df'.")
    exit()

In [19]:
# Choose your desired output format
output_choice = input("Export to (csv, json, sql): ").strip().lower()

Export to (csv, json, sql): sql


In [21]:
# Export based on selected output format
if output_choice == 'csv':
    df_to_export.to_csv('output.csv', index=False)
    print("Data exported to 'output.csv'")

elif output_choice == 'json':
    df_to_export.to_json('output.json', orient='records', lines=True)
    print("Data exported to 'output.json'")

elif output_choice == 'sql':
    # Establish a connection to SQLite database
    conn = sql.connect('output.db')
    df_to_export.to_sql('data', conn, if_exists='replace', index=False)
    conn.close()
    print("Data exported to 'output.db' (table: 'data')")

    # Read the SQL table and print its contents
    conn = sql.connect('output.db')
    sql_table = pd.read_sql('SELECT * FROM data', conn)
    print("\nContents of the SQL table 'data':")
    print(sql_table)
    conn.close()

else:
    print("Invalid output choice. Please select 'csv', 'json', or 'sql'.")

Data exported to 'output.db' (table: 'data')

Contents of the SQL table 'data':
      Month  Day  Port Code            Port Name     Border  \
0        12    1       2310                 Roma  US-Mexico   
1        12    1       2302              Del Rio  US-Mexico   
2        11    1       2310                 Roma  US-Mexico   
3        10    1       2601              Douglas  US-Mexico   
4         8    1        206        Beecher Falls  US-Canada   
...     ...  ...        ...                  ...        ...   
9082     11    1       3803   Sault Sainte Marie  US-Canada   
9083      3    1       3401              Pembina  US-Canada   
9084      3    1       3604  International Falls  US-Canada   
9085      9    1       3416                Maida  US-Canada   
9086      5    1       2506            Otay Mesa  US-Mexico   

       Mode of Transportation  Total Count  
0                       Buses           46  
1                      Trucks         6552  
2                      Truck